In [7]:
from data_loader_asp import AspectDataset, get_dataset, MamsAspectDataset, DataLoaderGen
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from prompts import prompt_dict
import numpy as np
# from evaluations_extracted_terms import infer_with_dataset
import pandas as pd
import ast
from selective_generation.semantic_enc import PhraseEncoder
import torch
# from matplotlib import pyplot as plt
import os

# Objectives Data Analysis Of labelled:
Are the text examples having no Aspect Terms(AT) are semantically different from the instances having atleast 1 AT.

### Experiment

$t_{x_{1}}, t_{x_{2}}... t_{x_{n}}$: Text inputs
1. Select all the text instances having ATs. 
  
  $$ds = ds_{AT>0} + ds_{AT=0}$$

2. Calculate vector representations (sentence embeddings) for all the text samples.

<!-- $f_{embed}(t_{x_i}) = V_{x_i} \forall (t_{x_i} \in ds)$ -->
3. Get average of the vector representation of the labelled dataset.
4. Calculate cosine similarity for all text, with the average vector representation calculated in 3.
6. Calculate similarity aggregations for $ds_{AT>0}, ds_{AT=0}$

In [8]:
LAPTOP_TRAIN = '/home/gauneg/llm_experiments/ds_for_generation/ft_LLM_gen/laptop_train.csv'
RESTAURANT_TRAIN = '/home/gauneg/llm_experiments/ds_for_generation/ft_LLM_gen/restaurant_train.csv'

laptop_df = pd.read_csv(LAPTOP_TRAIN)
restaurant_df = pd.read_csv(RESTAURANT_TRAIN)

# # datadict = {'train': df_train.values, 'valid': df_valid.values, 'combined': np.vstack((df_train.values, df_valid.values))}
# datadict = {'laptop_gold': laptop_df.values, 'restaurant_gold':restaurant_df.values}
# tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

def split_sim_calc(df, sim_arr):
    lab_text = []
    un_lab_text = []
    p_enc = PhraseEncoder()
    p_enc.pre_calc_average(sim_arr)
    for text, terms in df.values:
        terms_parsed = ast.literal_eval(terms)
        if len(terms_parsed)>0:
            lab_text.append([text, terms, p_enc.calculate_cosine_sim_avg(text)])
        else:
            un_lab_text.append([text, terms, p_enc.calculate_cosine_sim_avg(text)])
    return lab_text, un_lab_text


In [9]:
# creating sim array, text used as ref for similarity

term_list = []
text_list = []
for text, terms in restaurant_df.values:
     terms_parsed = ast.literal_eval(terms)
     for term in terms_parsed:
          term_list.append(term[0].lower())
     if len(terms_parsed)>0:
          text_list.append(text.lower())

In [7]:
# 
# lab_text, un_lab_text = split_sim_calc(restaurant_df, list(set(term_list)))
# un_lab_df = pd.DataFrame(un_lab_text, columns=['text', 'terms', 'sim_score'])
# lab_df = pd.DataFrame(lab_text, columns=['text', 'terms', 'sim_score'])
# ux = un_lab_df['sim_score']
# lx = lab_df['sim_score']
# print('unlabelled',   ux.mean(), ux.median())
# print('labelled',   lx.mean(), lx.median())

20it [00:00, 70.50it/s]


unlabelled 0.22226852 0.21172869
labelled 0.30443174 0.32107848


In [10]:
lap_dir = '/home/gauneg/llm_experiments/ds_for_generation/selective_gen_laptop'
res_dir = '/home/gauneg/llm_experiments/ds_for_generation/selective_gen_restaurant'
lab_text, un_lab_text = split_sim_calc(restaurant_df, text_list)
un_lab_df = pd.DataFrame(un_lab_text, columns=['text', 'terms', 'sim_score'])
lab_df = pd.DataFrame(lab_text, columns=['text', 'terms', 'sim_score'])
ux = un_lab_df['sim_score']
lx = lab_df['sim_score']
print('unlabelled',   ux.mean(), ux.median())
print('labelled',   lx.mean(), lx.median())


24it [00:00, 39.59it/s]


unlabelled 0.39309788 0.39394715
labelled 0.46724558 0.47146356


#### To check if lab_sim better than text_sim
The objective of similarity score calculation is to **assign higher score to the unlabelled text, which is similar to text with labels.** 

Calculate similarity for:
1. All sim with average of unique aspect terms in labs.
2. All sim with average of all aspect terms in labs
3. All sim with average of the text of datapoints having labelled aspect terms.


| Sno. | Method                   | Mean sim of unlabelled(L) | Mean sim of labelled(L) | Mean sim of unlabelled(R) | Mean sim of labelled (R) |
| ---- | ------------------------ | ----------------------- | ------------------------- | ----------------------- | -------------------------- |
| 1    | Unique Asp Terms         |    0.247                |    0.235                  |    0.222                |        0.304               |
| 2    | All Asp Terms            |    0.233                |    0.230                  |    0.202                |        0.277               |
| 3    | Text where asp in lab    |    0.389                |    0.434                  |    0.339                |        0.475               |

The evidence points and justifies, the use of text for similarity measure.

In [11]:
un_lab_df.to_csv(os.path.join(lap_dir, 'res_llm_unlab.csv'), index=False)
lab_df.to_csv(os.path.join(lap_dir, 'res_llm_lab.csv'), index=False)

In [3]:
from selective_generation.semantic_enc import PhraseEncoder
penc = PhraseEncoder()

In [8]:
rep = penc.encode_sentences(['this that them'])
rep[-1].shape

torch.Size([1, 384])

In [1]:
import os
import argparse
msg = "Adding description"


os.environ['CUDA_VISIBLE_DEVICES'] = "1"

from asp_pol_dataset import AspectDataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from self_eval import calc_labs
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import pandas as pd
from prompts import prompt_dict
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
# from evaluations_extracted_terms import evaluate_with_dataset, log_res
from datetime import datetime

def convert_ids_text(id_arr, tokenizer):
    # print(id_arr)
    toks = tokenizer.convert_ids_to_tokens(id_arr, skip_special_tokens=True)
    return tokenizer.convert_tokens_to_string(toks).strip()

LAPTOP_GOLD_PATH = '/home/gauneg/hester_letter/all_combined_ds/aspect_sentiment_polarity/laptop_2014/test.csv'
RESTAURANT_GOLD_PATH = '/home/gauneg/hester_letter/all_combined_ds/aspect_sentiment_polarity/restaurant_2014/test.csv'
BATCH_SIZE = 32
df_lap = pd.read_csv(LAPTOP_GOLD_PATH)
df_res = pd.read_csv(RESTAURANT_GOLD_PATH)


tokenizer = AutoTokenizer.from_pretrained(f'google/flan-t5-large')
model = AutoModelForSeq2SeqLM.from_pretrained(f'google/flan-t5-large')
datadict = {
    "laptop_gold": DataLoader(AspectDataset(df_lap.values, tokenizer), batch_size=BATCH_SIZE), 
    "restaurant_gold":  DataLoader(AspectDataset(df_res.values, tokenizer), batch_size=BATCH_SIZE)
    }


def infer_res(dataloader, model, tokenizer):
    fin_lab_pred = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    for batch in tqdm(dataloader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        gold_labels = batch['labels']
        pred_labels = model.generate(input_ids.to(device))
        for i in range(input_ids.shape[0]):
            input_txt = convert_ids_text(input_ids[i], tokenizer)
            pred_txt = convert_ids_text(pred_labels[i], tokenizer)
            gold_txt = convert_ids_text(gold_labels[i], tokenizer)
            fin_lab_pred.append([input_txt, pred_txt, gold_txt])    
    return fin_lab_pred 


/home/gauneg/anaconda3/envs/torch_test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for k, dloader in datadict.items():
    metric_arr = []

    
    res_arr = infer_res(dloader, model, tokenizer)
    df_res = pd.DataFrame(res_arr, columns=['text', 'pred_y', 'true_y'])
    metric_arr.append([k, 
                    precision_score(df_res['true_y'], df_res['pred_y'], zero_division=0, average='weighted'), 
                    recall_score(df_res['true_y'], df_res['pred_y'], zero_division=0, average='weighted'), 
                    f1_score(df_res['true_y'], df_res['pred_y'], zero_division=0, average='weighted'),
                    precision_score(df_res['true_y'], df_res['pred_y'], zero_division=0, average='macro'),
                    recall_score(df_res['true_y'], df_res['pred_y'], zero_division=0, average='macro'), 
                    f1_score(df_res['true_y'], df_res['pred_y'], zero_division=0, average='macro'),
                    ])
    print(metric_arr)

  0%|          | 0/21 [00:00<?, ?it/s]/home/gauneg/anaconda3/envs/torch_test/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 21/21 [02:05<00:00,  5.98s/it]


[['laptop_gold', 0.6891977807435996, 0.6972477064220184, 0.6015487827327012, 0.5199644581049252, 0.48545854056768295, 0.41650798177103965]]


  0%|          | 0/36 [00:00<?, ?it/s]/home/gauneg/anaconda3/envs/torch_test/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 36/36 [03:35<00:00,  5.98s/it]

[['restaurant_gold', 0.8142065371292829, 0.7839506172839507, 0.7170422616825931, 0.6206317976832612, 0.47586342229199374, 0.4337858946080941]]


In [ ]:
[['laptop_gold_small', 0.5350237852531431, 0.6269113149847095, 0.5444088228956129, 0.341468253968254, 0.4513910842956671, 0.35502797404115227]]
[['restaurant_gold_small', 0.6973597343675738, 0.7486772486772487, 0.68581841228952, 0.4274241229798975, 0.4649234693877551, 0.3929289480213001]]

[['laptop_gold_base', 0.6661623101947468, 0.6880733944954128, 0.6625595767091559, 0.4669860926826871, 0.5092119757956064, 0.47138912288999]]
[['restaurant_gold_base', 0.7372249635046139, 0.753968253968254, 0.7379939904631262, 0.4649688527096232, 0.5024038461538461, 0.47276313303099016]]

[['laptop_gold_large', 0.6891977807435996, 0.6972477064220184, 0.6015487827327012, 0.5199644581049252, 0.48545854056768295, 0.41650798177103965]]
[['restaurant_gold_large', 0.8142065371292829, 0.7839506172839507, 0.7170422616825931, 0.6206317976832612, 0.47586342229199374, 0.4337858946080941]]